In [1]:
import hydra
import numpy as np
import pytorch_lightning as pl
import torch
from omegaconf import DictConfig, OmegaConf
from transformers import AutoTokenizer

from data import PunctuationDataModule, PunctuationInferenceDataset
import os
from models import PunctuationDomainModel

from nemo.utils.exp_manager import exp_manager
from time import time
from pytorch_lightning.callbacks import ModelCheckpoint

import atexit
from copy import deepcopy
import snoop
snoop.install()

from hydra.experimental import initialize, initialize_config_module, initialize_config_dir, compose
hydra.core.global_hydra.GlobalHydra.instance().clear()
# folder="TEDend2021-02-11_07-57-33"
# folder="TEDstart2021-02-11_07-55-58"
# folder="TEDnone2021-02-11_10-14-34"
# folder="bilstmcrfbs82021-02-15_10-38-18"
# folder="bilstmnocrf2021-02-15_13-11-35"
folder="bilstm1fcl2021-02-16_13-11-12"
# initialize(config_path="../Punctuation_with_Domain_discriminator/"+folder) 
initialize()
!pwd
cfg=compose(
#     config_name="hparams.yaml", 
    config_name="config.yaml",
)
cfg.model.punct_label_ids=OmegaConf.create(sorted(cfg.model.punct_label_ids))
labels_to_ids = {_[1]:_[0] for _ in enumerate(cfg.model.punct_label_ids)}
ids_to_labels = {_[0]:_[1] for _ in enumerate(cfg.model.punct_label_ids)}
# model = PunctuationDomainModel.load_from_checkpoint( #TEDend2021-02-11_07-57-33  # TEDstart2021-02-11_07-55-58
#     checkpoint_path=f"/home/nxingyu/project/Punctuation_with_Domain_discriminator/{folder}/checkpoints/Punctuation_with_Domain_discriminator-last.ckpt")
# trainer = pl.Trainer(**cfg.trainer)

/home/nxingyu/project/experiment


In [2]:
ids_to_labels

{0: '',
 1: '!',
 2: '#',
 3: ',',
 4: '-',
 5: '.',
 6: ':',
 7: ';',
 8: '?',
 9: '—',
 10: '…'}

In [2]:

queries = [
    'Hooooooo!',
    'what can i do for you today',
    'how are you',
    ' in guadeloupe or marti nique it also brin gs into ques tion budgetary po licy because the europe an union is after all making a present of ecu 1.9 billion to three multinationals where are the financial interests of the european union firstly development policy in africa in any case in the acp countries employ ment policy in madeira the canaries guadel oupe martinique and crete regional pol icy in the ultra-peripheral areas human rights which mr barthet mayer mentioned earlier since dollar bananas are after all slavery bana nas the product of human exploitation by three multinat ionals payments of ecu 50 per mon th instead of ecu 50 per day',
    '''Plans for this weekend include turning wine into water.
The small white buoys marked the location of hundreds of crab pots.
He said he was not there yesterday; however, many people saw him there.
Today arrived with a crash of my car through the garage door.
The lyrics of the song sounded like fingernails on a chalkboard.
The Guinea fowl flies through the air with all the grace of a turtle.
They ran around the corner to find that they had traveled back in time.''',
    'good morning everyone how have your weekends been its a really great day thank you',
    'first of all i too agree that tourism related action must include employment training and education as you know after the european conference on tourism and employment in luxemborg we set up a high-level group whose mission was to examine how best tourism could contribute towards employment the first stage',
    "Ummm Im not really sure can you check with mr john instead thanks"
]
inference_results = model.add_punctuation(queries)
inference_results

[' hooooooo.                                                                                                                           ',
 ' what can i do for you today?                                                                                                                        ',
 ' how are you.                                                                                                                            ',
 ' in guadeloupe, or marti- nique, it also brin- gs into ques- tion budgetary po- licy. because the europe an union is after all, making a present of ecu 1, 9 billion to three multinationals. where are the financial interests of the european union? firstly, development policy in africa— in any case, in the acp countries employ ment policy in madeira, the canaries, guadel oupe, martinique and crete, regional pol- icy, in the ultra peripheral areas— human rights, which mr barthet mayer mentioned earlier, since dollar bananas are after all, slavery bana nas, the 

In [16]:

queries = [
    'Hooooooo!',
    'what can i do for you today',
    'how are you',
    ' in guadeloupe or marti nique it also brin gs into ques tion budgetary po licy because the europe an union is after all making a present of ecu 1.9 billion to three multinationals where are the financial interests of the european union firstly development policy in africa in any case in the acp countries employ ment policy in madeira the canaries guadel oupe martinique and crete regional pol icy in the ultra-peripheral areas human rights which mr barthet mayer mentioned earlier since dollar bananas are after all slavery bana nas the product of human exploitation by three multinat ionals payments of ecu 50 per mon th instead of ecu 50 per day',
    '''Plans for this weekend include turning wine into water.
The small white buoys marked the location of hundreds of crab pots.
He said he was not there yesterday; however, many people saw him there.
Today arrived with a crash of my car through the garage door.
The lyrics of the song sounded like fingernails on a chalkboard.
The Guinea fowl flies through the air with all the grace of a turtle.
They ran around the corner to find that they had traveled back in time.''',
    'good morning everyone how have your weekends been its a really great day thank you',
    'first of all i too agree that tourism related action must include employment training and education as you know after the european conference on tourism and employment in luxemborg we set up a high-level group whose mission was to examine how best tourism could contribute towards employment the first stage',
    "Ummm Im not really sure can you check with mr john instead thanks"
]
inference_results = model.add_punctuation(queries)
inference_results

[' hooooooo.                                                                                                                           ',
 ' what can i do for you? today                                                                                                                        ',
 ' how are you?                                                                                                                            ',
 ' in guadeloupe or marti nique, it also brin gs into ques tion budgetary po- licy, because the europe, an union is after all, making a present of ecu 1, 9 billion to three multinationals. where are the financial interests of the european union? firstly, development policy in africa in any case, in the acp countries, employ ment policy in madeira, the canaries, guadel oupe, martinique and crete, regional pol- icy in the ultra- peripheral areas— human rights, which mr barthet mayer mentioned earlier, since dollar bananas are after all, slavery, bana nas the pro

In [4]:

queries = [
    'Hooooooo!',
    'what can i do for you today',
    'how are you',
    ' in guadeloupe or marti nique it also brin gs into ques tion budgetary po licy because the europe an union is after all making a present of ecu 1.9 billion to three multinationals where are the financial interests of the european union firstly development policy in africa in any case in the acp countries employ ment policy in madeira the canaries guadel oupe martinique and crete regional pol icy in the ultra-peripheral areas human rights which mr barthet mayer mentioned earlier since dollar bananas are after all slavery bana nas the product of human exploitation by three multinat ionals payments of ecu 50 per mon th instead of ecu 50 per day',
    '''Plans for this weekend include turning wine into water.
The small white buoys marked the location of hundreds of crab pots.
He said he was not there yesterday; however, many people saw him there.
Today arrived with a crash of my car through the garage door.
The lyrics of the song sounded like fingernails on a chalkboard.
The Guinea fowl flies through the air with all the grace of a turtle.
They ran around the corner to find that they had traveled back in time.''',
    'good morning everyone how have your weekends been its a really great day thank you',
    'first of all i too agree that tourism related action must include employment training and education as you know after the european conference on tourism and employment in luxemborg we set up a high-level group whose mission was to examine how best tourism could contribute towards employment the first stage'
]
inference_results = model.add_punctuation(queries)
inference_results

[' hooooooo.                                                                                                                           ',
 ' what can i do for you? today                                                                                                                        ',
 ' how are you?                                                                                                                            ',
 ' in guadeloupe or marti nique, it also brin gs into ques tion budgetary po- licy because the europe, an union is after all, making a present of ecu 1, 9 billion to three multinationals. where are the financial interests of the european union? firstly, development policy in africa in any case, in the acp countries, employ ment policy in madeira, the canaries, guadel oupe, martinique and crete, regional pol- icy in the ultra- peripheral areas— human rights, which mr barthet mayer mentioned earlier, since dollar bananas are after all, slavery, bana nas the prod

In [9]:

queries = [
    'Hooooooo!',
    'what can i do for you today',
    'how are you',
    'firstly development policy in africa in any case in the acp countries employment policy in madeira the canaries guadeloupe martinique and crete regional policy in the ultra-peripheral areas human rights which mr barthet-mayer mentioned earlier since dollar bananas are after all slavery bananas the product of human exploitation by three multinationals payments of ecu 50 per month instead of ecu 50 per day in guadeloupe or martinique it also brings into question budgetary policy because the european union is after all making a present of ecu 1.9 billion to three multinationals where are the financial interests of the european union',
    '''Plans for this weekend include turning wine into water.
The small white buoys marked the location of hundreds of crab pots.
He said he was not there yesterday; however, many people saw him there.
Today arrived with a crash of my car through the garage door.
The lyrics of the song sounded like fingernails on a chalkboard.
The Guinea fowl flies through the air with all the grace of a turtle.
They ran around the corner to find that they had traveled back in time.''',
    'good morning everyone how have your weekends been its a really great day thank you',
    'first of all i too agree that tourism related action must include employment training and education as you know after the european conference on tourism and employment in luxemborg we set up a high-level group whose mission was to examine how best tourism could contribute towards employment the first stage'
]
inference_results = model.add_punctuation(queries)
inference_results

[' hooooooo,                                                                                                                           ',
 ' what can i do for you today?                                                                                                                        ',
 ' ? how are you?                                                                                                                            ',
 ' firstly, development policy in africa. in any case, in the acp, countries, employment policy in madeira, the canaries— guadeloupe, martinique and crete, regional policy in the ultra- peripheral areas— human rights, which mr barthet mayer, mentioned earlier, since dollar bananas are after all, slavery. bananas, the product of human exploitation by three multinationals, payments of ecu 50 per month, instead of ecu 50 per day in guadeloupe or martinique. it also brings into question budgetary policy. because the european union is after all, making a present 

In [20]:
# cfg.model.punct_label_ids=OmegaConf.create(sorted(cfg.model.punct_label_ids))
# labels_to_ids = {_[1]:_[0] for _ in enumerate(cfg.model.punct_label_ids)}

queries = [
    'Hooooooo!',
    'what can i doo for you? tooday?',
    'how are you',
    'good morning everyone how have your weekends been its a really great day'
]
cfg.model.punct_label_ids=OmegaConf.create(sorted(cfg.model.punct_label_ids))
labels_to_ids = {_[1]:_[0] for _ in enumerate(cfg.model.punct_label_ids)}
print(labels_to_ids)
ds=PunctuationInferenceDataset(tokenizer=AutoTokenizer.from_pretrained('google/electra-small-discriminator'), queries=queries, max_seq_length=128, punct_label_ids=labels_to_ids, degree=0, attach_label_to_end=None)
ds[0]

{'': 0, '!': 1, '#': 2, ',': 3, '-': 4, '.': 5, ':': 6, ';': 7, '?': 8, '—': 9, '…': 10}


{'input_ids': tensor([[  101,  7570,  9541,  9541,  9541,   102,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,    

In [18]:
sorted(['', '!', '#', ',', '-', '.', ':', ';', '?', '—', '…','#'])

['', '!', '#', '#', ',', '-', '.', ':', ';', '?', '—', '…']